# ELYZA-tasks-100 による性能評価

In [ ]:
!pip install accelerate datasets jsonlines

## ELYZA-tasks-100 データセットのダウンロード

モデルのダウンロードには 🤗datasets ライブラリを用いる。

データは `/content/dataset.jsonl` に保存する。

In [ ]:
import json
from pathlib import Path

from datasets import load_dataset

# Load the dataset
ds = load_dataset("elyza/ELYZA-tasks-100")

# Function to convert dataset to JSONL format and print
def dataset_to_jsonl(dataset, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        for entry in dataset:
            # Construct JSON object
            json_obj = {
                "input_text": entry['input'],
                "output_text": entry['output'],
                "eval_aspect": entry['eval_aspect']
            }

            # Write JSON object to file in JSONL format
            json_str = json.dumps(json_obj, ensure_ascii=False)
            file.write(json_str + '\n')

# Convert and write the dataset to a file in JSONL format
path_jsonl = Path("/content/dataset.jsonl")

if (path_jsonl.parent is not None) and (not path_jsonl.parent.exists()):
    path_jsonl.parent.mkdir(parents=True, exist_ok=True)

dataset_to_jsonl(ds["test"], path_jsonl)


## 評価

ELYZA-tasks-100 データセットを用いて、指定したモデルの評価を実行する。

### ハイパーパラメータについて

ハイパーパラメータは 🤗HuggingFace の Code Snippet のとおりとする（航海者が指定したハイパーパラメータを、チューン済みとみなす）。
ただし、長文の回答を要求する設問もあるため、`max_new_tokens` は `1024` に統一する。

In [ ]:
import json
import jsonlines
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Setup model and tokenizer
model_name = "rinna/youri-7b-chat"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, low_cpu_mem_usage=True, device_map="auto")

def format_prompt(input_text):
    # ✋プロンプトは、モデルに合わせて調整（HuggingFace のモデルカード参照）
    prompt_template = """設定: あなたは役立つアシスタントです。ユーザの質問に答えてください。\nユーザー: {input}\nシステム: """
    return prompt_template.format(input=input_text)

def generate_text(input_text):
    formatted_prompt = format_prompt(input_text)
    input_ids = tokenizer.encode(
        formatted_prompt,
        add_special_tokens=False,
        return_tensors="pt"
    )

    # Set seed for reproducibility
    seed = 42
    torch.manual_seed(seed)

    # ✋Hparamsは、モデルに合わせて調整（HuggingFace のモデルカード参照）
    tokens = model.generate(
        input_ids.to(device=model.device),
        max_new_tokens=1024,
        do_sample=True,
        temperature=0.5,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

    # Remove the input tokens from the generated tokens before decoding
    output_tokens = tokens[0][len(input_ids[0]):]
    return tokenizer.decode(output_tokens, skip_special_tokens=True)

def process_dataset(input_file, output_file):
    # Read dataset and generate predictions
    with jsonlines.open(input_file) as reader, jsonlines.open(output_file, mode='w') as writer:
        for obj in reader:
            prompt = obj['input_text']
            generated_text = generate_text(prompt)
            print(generated_text)
            writer.write({"pred": generated_text})

# Process the dataset
input_dataset = '/content/dataset.jsonl'
output_predictions = '/content/preds.jsonl'
process_dataset(input_dataset, output_predictions)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


以下は、仕事の熱意を取り戻すためのアイデアです。

- その仕事の意味を考えてみてください。
- その仕事で何を達成したいのか考えてみてください。
- その仕事におけるあなたの貢献を考えてみてください。
- その仕事のやり方の変化を評価してみてください。
- その仕事におけるあなたの成果を考えてみてください。


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


ある日、クマはアザラシに出会いました。アザラシは海で泳いでいました。クマはアザラシの近くに座り、アザラシと話をしました。アザラシはとても楽しく、クマと一緒にいることが大好きでした。その後、アザラシはクマと一緒に家に帰りました。

アザラシはクマの家に滞在し、クマと一緒に過ごしました。アザラシはとても楽しく、クマと一緒にいることが大好きでした。その後、アザラシはクマと一緒に家に帰りました。

アザラシはクマと一緒にいることが大好きで、クマもアザラシと一緒にいることが大好きでした。


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

花粉症でも春を楽しむコツは、花粉症対策をすること、花粉症の症状を軽減すること、春の楽しみを見つけることです。


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1. 立派です。
2. 知識
3. 知識
4. 賞賛
5. 味付け


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


手紙の主は、主人公に秘密の世界を見せるつもりだった。
1. 「果物だけではなく、人生も同じです。協力しなくては、全てをこなすことはできません。」
2. 老人は、広い畑で色々な果物を育てていました。
3. ある日、孫が老人に『こんなに広い畑で、全ての果物を一人で育てるのは大変そうですね』と尋ねました。
4. 昔、あるところに老人と孫がいました。
5. 老人は微笑みながら次のように答えました。
